# Debugging and Analyzing Data from Arize Platform



Use this template to explore, analyze, and debug using data from the Arize platform. It takes in the data export URL, which you enter below, and produces a clean pandas dataframe that can be used for analysis.


 ***Note: Make a copy of this notebook to allow edits***


## Setting up the dataframe

Import libraries and define some helper functions.

In [ ]:
import json
import pandas as pd
import urllib.request


def get_value(d):
    for k, v in d.items():
        if k == "int":
            return int(v)
        if k == "double":
            return float(v)
        if k == "string":
            return str(v)
        return v
    return None


def clean_up_dict_values(dict_to_clean):
    for key in dict_to_clean:
        if type(dict_to_clean[key]) == dict:
            dict_to_clean[key] = get_value(dict_to_clean[key])

**Edit paramaters** with your export url and desired file preferences.

In [ ]:
# Add the URL to your file (provided by Arize) here
arize_ui_url = "YOUR_DATA_EXPORT_URL"
file_name = "downloaded_data.json"

Retrieve data from either the url or locally (if stored). Follow prompt instructions for authorization.



In [ ]:
urllib.request.urlretrieve(arize_ui_url, file_name)

Set up dataframe with the exported data.

In [ ]:
# construct the formatted dataframe in this dictionary
data_frame_dict = {}

# open up the json file
with open(file_name) as fp:

    # read the data point into a dictionary
    line = fp.readline()
    index = 0

    while line:

        formatted_data_point = {}
        data_point = json.loads(line)

        # prediction
        prediction_dict = data_point["prediction"]

        formatted_data_point["timestamp"] = prediction_dict["timestamp"]
        formatted_data_point["modelVersion"] = prediction_dict["modelVersion"]
        formatted_data_point["predictionId"] = data_point["predictionId"]


        # features
        if "features" in prediction_dict:

            features = prediction_dict["features"]
            clean_up_dict_values(features)
            for k in features:
                formatted_data_point[k] = features[k]
            del prediction_dict["features"]

        # score categorical models are structured differently
        if "scoreCategorical" in prediction_dict["label"]:
            prediction = prediction_dict["label"]["scoreCategorical"]["scoreCategory"][
                "category"
            ]
            if "score" in prediction_dict["label"]["scoreCategorical"]["scoreCategory"]:
                score = float(
                    prediction_dict["label"]["scoreCategorical"]["scoreCategory"][
                        "score"
                    ]
                )
            else:
                score = 0

            formatted_data_point["prediction_score"] = score
            formatted_data_point["prediction"] = prediction
        else:
            clean_up_dict_values(prediction_dict)
            prediction = prediction_dict["label"]
            formatted_data_point["prediction"] = prediction

        # actual
        if "actual" in data_point:
            actual_dict = data_point["actual"]

            # score categorical models are structured differently
            if "scoreCategorical" in actual_dict["label"]:
                actual = actual_dict["label"]["scoreCategorical"]["scoreCategory"][
                    "category"
                ]
                if "score" in actual_dict["label"]["scoreCategorical"]["scoreCategory"]:
                    score = float(
                        actual_dict["label"]["scoreCategorical"]["scoreCategory"][
                            "score"
                        ]
                    )
                else:
                    score = 0

                formatted_data_point["actual_score"] = score
                formatted_data_point["actual"] = actual
            else:
                clean_up_dict_values(actual_dict)
                actual = actual_dict["label"]
                formatted_data_point["actual"] = actual

        # add to new dataframe dict
        data_frame_dict[index] = formatted_data_point

        line = line = fp.readline()
        index += 1


prediction_df = pd.DataFrame(data_frame_dict)
prediction_df = prediction_df.transpose()
# Clean up - type timestamp to correct column type
prediction_df["timestamp"] = pd.to_datetime(prediction_df["timestamp"])
prediction_df["date_string"] = prediction_df.timestamp.dt.strftime("%Y-%m-%d")

Now the data is ready to be explored. Take a look at how it's formatted in the dataframe.

In [ ]:
prediction_df.head()

## Examples of breaking down the data

### Count of prediction and actuals

In [ ]:
# Ability to look at mean of prediction and actuals
# If you are slicing on features in the platform this gives some examples how to slice on the same feature

# Note this will not work in classification models where the predictions are True/False
"""
print(prediction_df['actual'].mean())
print(prediction_df[(prediction_df['modelVersion'] == '1.0') ]['prediction'].mean())
print(prediction_df[(prediction_df['modelVersion'] == '1.0') & (prediction_df.date_string > "2021-03-20")]['prediction'].mean())
""";

### MSE and other metrics

In [ ]:
# Note this will not work in classification models
"""
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
print(mean_absolute_error(prediction_df['actual'], prediction_df['prediction']))
slice_grade_3 = prediction_df[(prediction_df.date_string > "2021-03-20") ]
print(mean_absolute_error(slice_grade_3['actual'], slice_grade_3['prediction']))
""";

### Grouping data

In [ ]:
# Group all the prediction data by the day they were made
"""
prediction_df.groupby(['date_string']).count()['prediction'].head()
""";

## Workspace

Expand this notebook as much as you need for your data digging needs

## Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

## Website
Visit Us At: https://arize.com/model-monitoring/

## Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
